In [4]:
# 03_experiments_mlp.ipynb applying imbalance tricks during experiments

# speed and gpu
import os
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/opt/cuda'
import json, numpy as np, matplotlib.pyplot as plt, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import mixed_precision
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

tf.config.optimizer.set_jit(True)
try: tf.config.experimental.enable_tensor_float_32_execution(True)
except: pass
mixed_precision.set_global_policy('mixed_float16')
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except: pass

# load meta and data
with open('../artifacts/metadata.json') as f: meta = json.load(f)
num_classes = len(meta['label_classes'])
X_train = np.load('../artifacts/X_train.npy')
X_val   = np.load('../artifacts/X_val.npy')
y_train = np.load('../artifacts/y_train.npy')
y_val   = np.load('../artifacts/y_val.npy')

# one hot labels
y_train_oh = keras.utils.to_categorical(y_train, num_classes)
y_val_oh   = keras.utils.to_categorical(y_val,   num_classes)

# scale with saved stats or compute here
X_mean = np.load('../artifacts/X_mean.npy') if os.path.exists('../artifacts/X_mean.npy') else X_train.mean(0)
X_std  = np.load('../artifacts/X_std.npy')  if os.path.exists('../artifacts/X_std.npy')  else X_train.std(0) + 1e-8
X_train_s = (X_train - X_mean) / X_std
X_val_s   = (X_val   - X_mean) / X_std
np.save('../artifacts/X_mean.npy', X_mean)
np.save('../artifacts/X_std.npy',  X_std)

# cast to float16 for speed
X_train_s = X_train_s.astype('float16', copy=False)
X_val_s   = X_val_s.astype('float16', copy=False)

# imbalance handling on train only
# 1 undersample huge classes a bit
rng = np.random.default_rng(42)
max_keep = 10000
keep_idx = []
for c in range(num_classes):
    idx = np.where(y_train == c)[0]
    if len(idx) > max_keep:
        idx = rng.choice(idx, max_keep, replace=False)
    keep_idx.append(idx)
keep_idx = np.concatenate(keep_idx)
X_train_us = X_train_s[keep_idx]
y_train_us = y_train[keep_idx]

# 2 oversample rare classes up to target
target_per_class = 5000
xs, ys = [], []
for c in range(num_classes):
    m = (y_train_us == c)
    x_c = X_train_us[m]
    y_c = keras.utils.to_categorical(y_train_us[m], num_classes)
    n = len(x_c)
    if n > 0 and n < target_per_class:
        add = rng.integers(0, n, target_per_class - n)
        x_c = np.concatenate([x_c, x_c[add]], 0)
        y_c = np.concatenate([y_c, y_c[add]], 0)
    xs.append(x_c); ys.append(y_c)
X_train_bal = np.concatenate(xs, 0)
y_train_bal = np.concatenate(ys, 0)

# shuffle
perm = rng.permutation(len(X_train_bal))
X_train_bal = X_train_bal[perm]
y_train_bal = y_train_bal[perm]

# 3 class weights on the balanced labels too
y_train_bal_labels = np.argmax(y_train_bal, 1)
classes = np.unique(y_train_bal_labels)
weights = compute_class_weight('balanced', classes=classes, y=y_train_bal_labels)
class_weights = {int(c): float(w) for c, w in zip(classes, weights)}

# datasets
BATCH = 1024
train_ds = tf.data.Dataset.from_tensor_slices((X_train_bal, y_train_bal)).cache().shuffle(10000).batch(BATCH).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val_s,   y_val_oh)).cache().batch(BATCH).prefetch(tf.data.AUTOTUNE)

os.makedirs('../artifacts/experiments', exist_ok=True)

# configs vary layers width act and dropout
configs = [
    {'name':'relu_1x64_do0',     'layers':[64],          'act':'relu',    'dropout':0.0},
    {'name':'relu_2x64_do03',    'layers':[64,64],       'act':'relu',    'dropout':0.3},
    {'name':'relu_3x128_do03',   'layers':[128,128,128], 'act':'relu',    'dropout':0.3},
    {'name':'tanh_1x128_do0',    'layers':[128],         'act':'tanh',    'dropout':0.0},
    {'name':'tanh_2x128_do02',   'layers':[128,128],     'act':'tanh',    'dropout':0.2},
    {'name':'sigmoid_2x64_do02', 'layers':[64,64],       'act':'sigmoid', 'dropout':0.2},
    {'name':'relu_1x256_do02',   'layers':[256],         'act':'relu',    'dropout':0.2},
    {'name':'tanh_3x256_do03',   'layers':[256,256,256], 'act':'tanh',    'dropout':0.3},
]

# model builder
def build_model(input_dim, cfg, num_classes):
    inp = keras.Input(shape=(input_dim,))
    x = inp
    for u in cfg['layers']:
        x = keras.layers.Dense(u, activation=cfg['act'])(x)
        if cfg['dropout'] > 0.0:
            x = keras.layers.Dropout(cfg['dropout'])(x)
    out = keras.layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    m = keras.Model(inp, out)
    opt = keras.optimizers.legacy.Adam(1e-3)
    m.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return m

# train and log
best_name = None
best_metric = -1.0
results = []

for i, cfg in enumerate(configs, 1):
    print(f'[{i}/{len(configs)}] {cfg["name"]}')
    model = build_model(X_train_s.shape[1], cfg, num_classes)
    hist = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=20,
        verbose=1,
        class_weight=class_weights
    )

    # plots
    acc_path  = f"../artifacts/experiments/{cfg['name']}_acc.png"
    loss_path = f"../artifacts/experiments/{cfg['name']}_loss.png"
    plt.figure(); plt.plot(hist.history['accuracy']); plt.plot(hist.history['val_accuracy']); plt.legend(['train','val']); plt.title(cfg['name']+' acc'); plt.xlabel('epoch'); plt.ylabel('accuracy'); plt.savefig(acc_path, dpi=150); plt.close()
    plt.figure(); plt.plot(hist.history['loss']); plt.plot(hist.history['val_loss']); plt.legend(['train','val']); plt.title(cfg['name']+' loss'); plt.xlabel('epoch'); plt.ylabel('loss'); plt.savefig(loss_path, dpi=150); plt.close()

    # val metrics macro f1
    yv_pred = model.predict(X_val_s, batch_size=4096).argmax(1)
    macro_f1 = f1_score(y_val, yv_pred, average='macro')
    val_acc  = float(np.mean(yv_pred == y_val))

    # save confusion matrix for val
    cm = confusion_matrix(y_val, yv_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=meta['label_classes'])
    disp.plot(xticks_rotation=90, cmap='Blues')
    plt.title(cfg['name']+' val confusion')
    plt.tight_layout()
    plt.savefig(f"../artifacts/experiments/{cfg['name']}_val_confusion.png", dpi=150)
    plt.close()

    # classification report to txt
    rep = classification_report(y_val, yv_pred, target_names=meta['label_classes'])
    with open(f"../artifacts/experiments/{cfg['name']}_val_report.txt",'w') as f: f.write(rep)

    results.append({'name':cfg['name'],'layers':cfg['layers'],'act':cfg['act'],'dropout':cfg['dropout'],'val_acc':val_acc,'macro_f1':float(macro_f1)})

    # choose best by macro f1 to be fair to rare classes
    if macro_f1 > best_metric:
        best_metric = macro_f1
        best_name = cfg['name']
        model.save('../artifacts/best_so_far.keras')
        with open('../artifacts/best_so_far_name.txt','w') as f: f.write(best_name)

# save summary
with open('../artifacts/experiments/results.json','w') as f: json.dump(results, f, indent=2)
print('best by macro f1', best_name, best_metric)
for r in sorted(results, key=lambda x: x['macro_f1'], reverse=True):
    print(r)


[1/8] relu_1x64_do0
Epoch 1/20
69/69 [==============================] - 1s 6ms/step - loss: 2.2219 - accuracy: 0.1665 - val_loss: 2.2260 - val_accuracy: 0.1358
Epoch 2/20
69/69 [==============================] - 1s 7ms/step - loss: 1.9701 - accuracy: 0.2479 - val_loss: 2.1117 - val_accuracy: 0.1898
Epoch 3/20
69/69 [==============================] - 0s 3ms/step - loss: 1.7789 - accuracy: 0.3017 - val_loss: 2.0028 - val_accuracy: 0.2346
Epoch 4/20
69/69 [==============================] - 0s 3ms/step - loss: 1.6391 - accuracy: 0.3342 - val_loss: 1.9162 - val_accuracy: 0.2662
Epoch 5/20
69/69 [==============================] - 0s 3ms/step - loss: 1.5426 - accuracy: 0.3669 - val_loss: 1.8538 - val_accuracy: 0.2895
Epoch 6/20
69/69 [==============================] - 0s 3ms/step - loss: 1.4740 - accuracy: 0.3860 - val_loss: 1.8229 - val_accuracy: 0.2981
Epoch 7/20
69/69 [==============================] - 0s 4ms/step - loss: 1.4230 - accuracy: 0.3943 - val_loss: 1.7878 - val_accuracy: 0.3131
